In [1]:
45

45

In [3]:
import os
from dotenv import load_dotenv

import pandas as pd
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()

c:\mscfy\anime\anienv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [5]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [6]:
import os
from langchain.chat_models import init_chat_model
model= init_chat_model("groq:llama-3.3-70b-versatile")
response =model.invoke("why am i studying at dot?")
response

AIMessage(content='It seems like you\'re studying at a place referred to as "DOT". Without more context, I can only make some educated guesses. Here are a few possibilities:\n\n1. **DOT as a university or college**: You might be studying at a university or college with the acronym "DOT". There are several institutions with similar names, such as the Department of Transportation (DOT) universities or colleges that have "DOT" in their name.\n2. **DOT as a training program**: You could be studying at a specialized training program or course with the name "DOT". This might be related to a specific field or industry, such as transportation, technology, or design.\n3. **DOT as a online learning platform**: It\'s also possible that "DOT" refers to an online learning platform or educational website that offers courses, tutorials, or degree programs.\n\nTo better understand why you\'re studying at DOT, could you please provide more context or information about what DOT stands for and what you\'

In [7]:

class AnimeDataLoader:
    def __init__(self, original_csv: str, processed_csv: str):
        self.original_csv = original_csv
        self.processed_csv = processed_csv

    def load_and_process(self):
        df = pd.read_csv(
            self.original_csv,
            encoding="utf-8",
            on_bad_lines="skip"
        ).dropna()

        required_cols = {"Name", "Genres", "sypnopsis"}
        if not required_cols.issubset(df.columns):
            raise ValueError("Missing required columns in CSV")

        df["combined_info"] = (
            "Title: " + df["Name"]
            + " Overview: " + df["sypnopsis"]
            + " Genres: " + df["Genres"]
        )

        df[["combined_info"]].to_csv(
            self.processed_csv,
            index=False,
            encoding="utf-8"
        )

        return self.processed_csv



In [ ]:
original_csv=("data/anime_with_synopsis.csv")
processed_csv=("data/anime_updated.csv")

data_loader =AnimeDataLoader(original_csv,processed_csv)
data_loader.load_and_process()




'data/anime_updated.csv'

In [11]:
class VectorStoreBuilder:
    def __init__(self, csv_path: str, persist_dir: str = "chroma_db"):
        self.csv_path = csv_path
        self.persist_dir = persist_dir
        self.embedding = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2"
        )

    def build_and_save_vectorstore(self):
        loader = CSVLoader(
            file_path=self.csv_path,
            encoding="utf-8",
            metadata_columns=[]
        )

        documents = loader.load()

        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=0
        )
        chunks = splitter.split_documents(documents)

        db = Chroma.from_documents(
            chunks,
            self.embedding,
            persist_directory=self.persist_dir
        )
        db.persist()

    def load_vector_store(self):
        return Chroma(
            persist_directory=self.persist_dir,
            embedding_function=self.embedding
        )


In [16]:
print("Building vector database...")

original_csv = "data/anime_with_synopsis.csv"
processed_csv = "data/anime_updated.csv"
persist_dir = "chroma_db"

loader = AnimeDataLoader(original_csv, processed_csv)
processed_csv_path = loader.load_and_process()

vector_builder = VectorStoreBuilder(
    processed_csv_path,
    persist_dir=persist_dir
)
vector_builder.build_and_save_vectorstore()

print("Vector database ready.")

Building vector database...
Vector database ready.


In [17]:
vector_builder = VectorStoreBuilder(
    csv_path="",
    persist_dir="chroma_db" 
    
)
retriver=vector_builder.load_vector_store().as_retriever()

C:\Users\JANHAVI\AppData\Local\Temp\ipykernel_228648\3258009850.py:32: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  return Chroma(


In [18]:
query="tell me about naruto"
retriver.invoke(query)

[Document(metadata={'row': 10, 'source': 'data/anime_updated.csv'}, page_content="combined_info: Title: Naruto Overview: oments prior to Naruto Uzumaki's birth, a huge demon known as the Kyuubi, the Nine-Tailed Fox, attacked Konohagakure, the Hidden Leaf Village, and wreaked havoc. In order to put an end to the Kyuubi's rampage, the leader of the village, the Fourth Hokage, sacrificed his life and sealed the monstrous beast inside the newborn Naruto. Now, Naruto is a hyperactive and knuckle-headed ninja still living in Konohagakure. Shunned because of the Kyuubi inside him, Naruto struggles to find his place in the village, while his burning desire to become the Hokage of Konohagakure leads him not only to some great new friends, but also some deadly foes. Genres: Action, Adventure, Comedy, Super Power, Martial Arts, Shounen"),
 Document(metadata={'source': 'data/anime_updated.csv', 'row': 10}, page_content="combined_info: Title: Naruto Overview: oments prior to Naruto Uzumaki's birth,

In [19]:
@tool
def anime_retriever_tool(query: str) -> str:
    """
    Use this tool to search the anime knowledge base.

    Always call this tool for anime-related questions such as:
    recommendations, similarity search, genres, or plot summaries.

    Input:
    - query: User's anime preference or question.

    Output:
    - Relevant anime information retrieved from the vector database.
    """
    docs = retriver.invoke(query)
    return "\n\n".join(doc.page_content for doc in docs)


In [20]:
model_with_tools=model.bind_tools([anime_retriever_tool])

In [21]:
messages=[
    {
        "role":"user",
        "content":"can you suggest something similar to naruto and also tell me joke"
    }
]

In [22]:
ai_msg=model_with_tools.invoke(messages)
messages.append(ai_msg)

 

In [23]:
messages

[{'role': 'user',
  'content': 'can you suggest something similar to naruto and also tell me joke'},
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'm6jet84q2', 'function': {'arguments': '{"query":"similar to Naruto"}', 'name': 'anime_retriever_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 340, 'total_tokens': 381, 'completion_time': 0.11099262, 'completion_tokens_details': None, 'prompt_time': 0.016382186, 'prompt_tokens_details': None, 'queue_time': 0.160504022, 'total_time': 0.127374806}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3272ea2d91', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c1354-5000-77e1-8642-c513ac6a49b0-0', tool_calls=[{'name': 'anime_retriever_tool', 'args': {'query': 'similar to Naruto'}, 'id': 'm6jet84q2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 340,

In [24]:
ai_msg.tool_calls

[{'name': 'anime_retriever_tool',
  'args': {'query': 'similar to Naruto'},
  'id': 'm6jet84q2',
  'type': 'tool_call'}]

In [25]:
for tool_call in ai_msg.tool_calls:
    tool_result=anime_retriever_tool.invoke(tool_call)
    messages.append(tool_result)

In [26]:
messages

[{'role': 'user',
  'content': 'can you suggest something similar to naruto and also tell me joke'},
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'm6jet84q2', 'function': {'arguments': '{"query":"similar to Naruto"}', 'name': 'anime_retriever_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 340, 'total_tokens': 381, 'completion_time': 0.11099262, 'completion_tokens_details': None, 'prompt_time': 0.016382186, 'prompt_tokens_details': None, 'queue_time': 0.160504022, 'total_time': 0.127374806}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3272ea2d91', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c1354-5000-77e1-8642-c513ac6a49b0-0', tool_calls=[{'name': 'anime_retriever_tool', 'args': {'query': 'similar to Naruto'}, 'id': 'm6jet84q2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 340,

In [27]:
final_response=model_with_tools.invoke(messages)
print(final_response.text)

Here's a joke for you: Why couldn't the ninja find his cat? Because it was purr-fectly hidden! 

If you like Naruto, you might enjoy other anime shows like "Bleach", "One Piece", "Blue Exorcist", or "My Hero Academia". These shows share similar themes of action, adventure, and self-discovery, and are likely to appeal to fans of the Naruto series.
